### A quick example on displaying detailed predictions from a model and calculating validation score
https://www.kaggle.com/slawekbiel/sartorius-vis/ script is used for visualisation. You need to do File->Add utility script to use it.

In [ ]:
!pip install ../input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron-05/whls 

In [ ]:
import torch
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog
import cv2
import pycocotools.mask as mask_util
import numpy as np


In [ ]:
from sartorius_vis import show_predictions, show_zoomed

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
cfg.MODEL.WEIGHTS = '../input/sartorius-models/model_2696.pth'  
cfg.TEST.DETECTIONS_PER_IMAGE = 1000
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

register_coco_instances('sartorius_val',{},'../input/sartorius-cell-instance-segmentation-coco/annotations_val.json', 
                        '../input/sartorius-cell-instance-segmentation/')

val_ds = DatasetCatalog.get('sartorius_val')

In [ ]:
##Helper functions
def read_target_masks(item):
    enc_targs = list(map(lambda x:x['segmentation'], item['annotations']))
    #enc_targs = mask_util.frPyObjects(enc_targs, 520, 704)
    tars = torch.tensor(mask_util.decode(enc_targs)).cuda()
    return tars.permute(2,0,1).bool()

def get_preds_and_tars(item):
    im = cv2.imread(item['file_name'])
    pred = predictor(im) 
    pred_masks = pred['instances'].pred_masks
    tars = read_target_masks(item)
    return item['file_name'], pred_masks, tars


def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ['annotations']))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)
def score_all():
    scores = []
    for item in val_ds:
        im =  cv2.imread(item['file_name'])
        pred = predictor(im)       
        
        sc = score(pred, item)
        scores.append(sc)
        
    return np.mean(scores)

## Show a sample cell with the score breakdwn

In [ ]:
show_predictions(*get_preds_and_tars(val_ds[0]))

## Lets zoom in to better view all the individual predicted cells

In [ ]:
show_zoomed(*get_preds_and_tars(val_ds[0]))

## Do the same to another file

In [ ]:
show_predictions(*get_preds_and_tars(val_ds[1]))

In [ ]:
show_zoomed(*get_preds_and_tars(val_ds[1]))

## Finally calculate score across all validation files:

In [ ]:
score_all()